In [ ]:
# 설치
! pip install smolagents
# 마지막 릴리스 대신 소스에서 설치하려면 위 명령을 주석 처리하고 다음 명령의 주석을 해제하십시오.
# ! pip install git+https://github.com/huggingface/smolagents.git

# 에이전트 - 가이드 투어

이 가이드 방문에서는 에이전트를 구축하고 실행하며 사용 사례에 더 적합하도록 사용자 지정하는 방법을 배웁니다.

## 에이전트 유형 선택: CodeAgent 또는 ToolCallingAgent

`smolagents`에는 [CodeAgent](https://huggingface.co/docs/smolagents/main/ko/reference/agents#smolagents.CodeAgent)와 [ToolCallingAgent](https://huggingface.co/docs/smolagents/main/ko/reference/agents#smolagents.ToolCallingAgent)라는 두 가지 에이전트 클래스가 있으며, 이는 에이전트가 도구와 상호 작용하는 방식에 대한 두 가지 다른 패러다임을 나타냅니다.
주요 차이점은 작업이 지정되고 실행되는 방식에 있습니다: 코드 생성 대 구조화된 도구 호출.

- [CodeAgent](https://huggingface.co/docs/smolagents/main/ko/reference/agents#smolagents.CodeAgent)는 도구 호출을 Python 코드 스니펫으로 생성합니다.
  - 코드는 로컬(잠재적으로 안전하지 않음) 또는 보안 샌드박스에서 실행됩니다.
  - 도구는 Python 함수(바인딩을 통해)로 노출됩니다.
  - 도구 호출 예시:
    ```py
    result = search_docs("프랑스의 수도는 어디인가요?")
    print(result)
    ```
  - 강점:
    - 매우 표현력이 뛰어남: 복잡한 논리 및 제어 흐름을 허용하고 도구를 결합, 반복, 변환, 추론할 수 있습니다.
    - 유연함: 가능한 모든 작업을 미리 정의할 필요 없이 동적으로 새 작업/도구를 생성할 수 있습니다.
    - 창발적 추론: 다단계 문제 또는 동적 논리에 이상적입니다.
  - 한계점
    - 오류 위험: 구문 오류, 예외를 처리해야 합니다.
    - 예측 가능성 낮음: 예기치 않거나 안전하지 않은 출력이 발생하기 쉽습니다.
    - 보안 실행 환경이 필요합니다.

- [ToolCallingAgent](https://huggingface.co/docs/smolagents/main/ko/reference/agents#smolagents.ToolCallingAgent)는 도구 호출을 구조화된 JSON으로 작성합니다.
  - 이는 많은 프레임워크(OpenAI API)에서 사용되는 일반적인 형식으로, 코드 실행 없이 구조화된 도구 상호 작용을 허용합니다.
  - 도구는 JSON 스키마로 정의됩니다: 이름, 설명, 매개변수 유형 등.
  - 도구 호출 예시:
    ```json
    {
      "tool_call": {
        "name": "search_docs",
        "arguments": {
          "query": "프랑스의 수도는 어디인가요?"
        }
      }
    }
    ```
  - 강점:
    - 신뢰할 수 있음: 환각이 발생하기 쉽지 않고 출력이 구조화되고 검증됩니다.
    - 안전함: 인수가 엄격하게 검증되어 임의의 코드가 실행될 위험이 없습니다.
    - 상호 운용 가능: 외부 API 또는 서비스에 쉽게 매핑할 수 있습니다.
  - 한계점:
    - 표현력 낮음: 결과를 동적으로 결합하거나 변환하거나 복잡한 논리 또는 제어 흐름을 수행하기 쉽지 않습니다.
    - 유연성 부족: 가능한 모든 작업을 미리 정의해야 하며 미리 정의된 도구로 제한됩니다.
    - 코드 합성 불가: 도구 기능으로 제한됩니다.

어떤 에이전트 유형을 사용해야 할까요:
- 다음과 같은 경우 [CodeAgent](https://huggingface.co/docs/smolagents/main/ko/reference/agents#smolagents.CodeAgent)를 사용하십시오:
  - 추론, 연결 또는 동적 구성이 필요합니다.
  - 도구는 결합할 수 있는 함수입니다(예: 구문 분석 + 수학 + 쿼리).
  - 에이전트가 문제 해결사 또는 프로그래머입니다.

- 다음과 같은 경우 [ToolCallingAgent](https://huggingface.co/docs/smolagents/main/ko/reference/agents#smolagents.ToolCallingAgent)를 사용하십시오:
  - 간단하고 원자적인 도구가 있습니다(예: API 호출, 문서 가져오기).
  - 높은 신뢰성과 명확한 검증을 원합니다.
  - 에이전트가 디스패처 또는 컨트롤러와 같습니다.

## CodeAgent

[CodeAgent](https://huggingface.co/docs/smolagents/main/ko/reference/agents#smolagents.CodeAgent)는 작업을 수행하고 해결하기 위해 Python 코드 스니펫을 생성합니다.

기본적으로 Python 코드 실행은 로컬 환경에서 수행됩니다.
호출할 수 있는 유일한 함수는 제공한 도구(특히 Hugging Face의 도구만 있는 경우)와 `print` 또는 `math` 모듈의 함수와 같은 미리 정의된 안전한 함수 집합이므로 이미 실행할 수 있는 항목이 제한되어 있으므로 안전해야 합니다.

Python 인터프리터는 기본적으로 안전한 목록 외부에서의 가져오기를 허용하지 않으므로 가장 명백한 공격은 문제가 되지 않아야 합니다.
[CodeAgent](https://huggingface.co/docs/smolagents/main/ko/reference/agents#smolagents.CodeAgent) 초기화 시 `additional_authorized_imports` 인수에 승인된 모듈을 문자열 목록으로 전달하여 추가 가져오기를 승인할 수 있습니다.

In [ ]:
model = InferenceClientModel()
agent = CodeAgent(tools=[], model=model, additional_authorized_imports=['requests', 'bs4'])
agent.run("https://huggingface.co/blog 페이지의 제목을 알려주시겠어요?")

또한 추가 보안 계층으로, 가져오기 목록 내에서 명시적으로 승인되지 않는 한 하위 모듈에 대한 액세스는 기본적으로 금지됩니다.
예를 들어, `numpy.random` 하위 모듈에 액세스하려면 `additional_authorized_imports` 목록에 `'numpy.random'`을 추가해야 합니다.
이는 `numpy.*`를 사용하여 승인할 수도 있으며, 이렇게 하면 `numpy`뿐만 아니라 `numpy.random`과 같은 하위 패키지 및 자체 하위 패키지도 허용됩니다.

> [!WARNING]
> LLM은 실행될 임의의 코드를 생성할 수 있습니다. 안전하지 않은 가져오기를 추가하지 마십시오!

실행은 불법적인 작업을 수행하려는 모든 코드에서 중지되거나 에이전트가 생성한 코드에 일반적인 Python 오류가 있는 경우 중지됩니다.

로컬 Python 인터프리터 대신 [E2B 코드 실행기](https://e2b.dev/docs#what-is-e2-b) 또는 Docker를 사용할 수도 있습니다. E2B의 경우 먼저 [`E2B_API_KEY` 환경 변수를 설정](https://e2b.dev/dashboard?tab=keys)한 다음 에이전트 초기화 시 `executor_type="e2b"`를 전달합니다. Docker의 경우 초기화 중에 `executor_type="docker"`를 전달합니다.


> [!TIP]
> 코드 실행에 대한 자세한 내용은 [이 튜토리얼](https://huggingface.co/docs/smolagents/main/ko/tutorials/secure_code_execution)을 참조하십시오.

### ToolCallingAgent

[ToolCallingAgent](https://huggingface.co/docs/smolagents/main/ko/reference/agents#smolagents.ToolCallingAgent)는 JSON 도구 호출을 출력하며, 이는 많은 프레임워크(OpenAI API)에서 사용되는 일반적인 형식으로, 코드 실행 없이 구조화된 도구 상호 작용을 허용합니다.

코드를 실행하지 않으므로 `additional_authorized_imports` 없이 [CodeAgent](https://huggingface.co/docs/smolagents/main/ko/reference/agents#smolagents.CodeAgent)와 거의 동일한 방식으로 작동합니다.

In [ ]:
from smolagents import ToolCallingAgent

agent = ToolCallingAgent(tools=[], model=model)
agent.run("https://huggingface.co/blog 페이지의 제목을 알려주시겠어요?")

## 에이전트 구축

최소한의 에이전트를 초기화하려면 최소한 다음 두 가지 인수가 필요합니다.

- `model`, 에이전트를 구동할 텍스트 생성 모델 - 에이전트는 단순한 LLM과 다르며 LLM을 엔진으로 사용하는 시스템이기 때문입니다. 다음 옵션 중 하나를 사용할 수 있습니다.
    - [TransformersModel](https://huggingface.co/docs/smolagents/main/ko/reference/models#smolagents.TransformersModel)은 미리 초기화된 `transformers` 파이프라인을 사용하여 `transformers`를 사용하여 로컬 컴퓨터에서 추론을 실행합니다.
    - [InferenceClientModel](https://huggingface.co/docs/smolagents/main/ko/reference/models#smolagents.InferenceClientModel)은 내부적으로 `huggingface_hub.InferenceClient`를 활용하며 Cerebras, Cohere, Fal, Fireworks, HF-Inference, Hyperbolic, Nebius, Novita, Replicate, SambaNova, Together 등 Hub의 모든 추론 공급자를 지원합니다.
    - [LiteLLMModel](https://huggingface.co/docs/smolagents/main/ko/reference/models#smolagents.LiteLLMModel)도 마찬가지로 [LiteLLM](https://docs.litellm.ai/)을 통해 100개 이상의 다양한 모델 및 공급자를 호출할 수 있습니다!
    - [AzureOpenAIServerModel](https://huggingface.co/docs/smolagents/main/ko/reference/models#smolagents.AzureOpenAIServerModel)을 사용하면 [Azure](https://azure.microsoft.com/ko-kr/products/ai-services/openai-service)에 배포된 OpenAI 모델을 사용할 수 있습니다.
    - [AmazonBedrockServerModel](https://huggingface.co/docs/smolagents/main/ko/reference/models#smolagents.AmazonBedrockServerModel)을 사용하면 [AWS](https://aws.amazon.com/bedrock/?nc1=h_ls)에서 Amazon Bedrock을 사용할 수 있습니다.
    - [MLXModel](https://huggingface.co/docs/smolagents/main/ko/reference/models#smolagents.MLXModel)은 [mlx-lm](https://pypi.org/project/mlx-lm/) 파이프라인을 만들어 로컬 컴퓨터에서 추론을 실행합니다.

- `tools`, 에이전트가 작업을 해결하는 데 사용할 수 있는 `Tools` 목록입니다. 빈 목록일 수 있습니다. 선택적 인수 `add_base_tools=True`를 정의하여 `tools` 목록 위에 기본 도구 상자를 추가할 수도 있습니다.

이 두 인수 `tools`와 `model`이 있으면 에이전트를 만들고 실행할 수 있습니다. [추론 공급자](https://huggingface.co/blog/inference-providers), [transformers](https://github.com/huggingface/transformers/), [ollama](https://ollama.com/), [LiteLLM](https://www.litellm.ai/), [Azure OpenAI](https://azure.microsoft.com/ko-kr/products/ai-services/openai-service), [Amazon Bedrock](https://aws.amazon.com/bedrock/?nc1=h_ls) 또는 [mlx-lm](https://pypi.org/project/mlx-lm/)을 통해 원하는 LLM을 사용할 수 있습니다.

<hfoptions id="Pick a LLM">
<hfoption id="추론 공급자">

추론 공급자는 인증을 위해 `HF_TOKEN`이 필요하지만 무료 HF 계정에는 이미 포함된 크레딧이 제공됩니다. PRO로 업그레이드하여 포함된 크레딧을 늘리십시오.

제한된 모델에 액세스하거나 PRO 계정으로 속도 제한을 높이려면 환경 변수 `HF_TOKEN`을 설정하거나 `InferenceClientModel` 초기화 시 `token` 변수를 전달해야 합니다. [설정 페이지](https://huggingface.co/settings/tokens)에서 토큰을 얻을 수 있습니다.

In [ ]:
from smolagents import CodeAgent, InferenceClientModel

model_id = "meta-llama/Llama-3.3-70B-Instruct" 

model = InferenceClientModel(model_id=model_id, token="<YOUR_HUGGINGFACEHUB_API_TOKEN>") # InferenceClientModel에 model_id를 전달하지 않고 기본 모델을 사용할 수 있습니다.
# 특정 공급자를 지정할 수도 있습니다. 예: provider="together" 또는 provider="sambanova"
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run(
    "피보나치 수열의 118번째 숫자를 알려주시겠어요?",
)

</hfoption>
<hfoption id="로컬 트랜스포머 모델">

In [ ]:
# !pip install smolagents[transformers]
from smolagents import CodeAgent, TransformersModel

model_id = "meta-llama/Llama-3.2-3B-Instruct"

model = TransformersModel(model_id=model_id)
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run(
    "피보나치 수열의 118번째 숫자를 알려주시겠어요?",
)

</hfoption>
<hfoption id="OpenAI 또는 Anthropic API">

`LiteLLMModel`을 사용하려면 환경 변수 `ANTHROPIC_API_KEY` 또는 `OPENAI_API_KEY`를 설정하거나 초기화 시 `api_key` 변수를 전달해야 합니다.

In [ ]:
# !pip install smolagents[litellm]
from smolagents import CodeAgent, LiteLLMModel

model = LiteLLMModel(model_id="anthropic/claude-3-5-sonnet-latest", api_key="YOUR_ANTHROPIC_API_KEY") # 'gpt-4o'를 사용할 수 있습니다.
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run(
    "피보나치 수열의 118번째 숫자를 알려주시겠어요?",
)

</hfoption>
<hfoption id="Ollama">

In [ ]:
# !pip install smolagents[litellm]
from smolagents import CodeAgent, LiteLLMModel

model = LiteLLMModel(
    model_id="ollama_chat/llama3.2", # 이 모델은 에이전트 동작에는 약간 약합니다.
    api_base="http://localhost:11434", # 필요한 경우 127.0.0.1:11434 또는 원격 open-ai 호환 서버로 바꿉니다.
    api_key="YOUR_API_KEY", # 필요한 경우 API 키로 바꿉니다.
    num_ctx=8192, # ollama 기본값은 2048이며 심각하게 실패합니다. 8192는 쉬운 작업에 적합하며 더 많을수록 좋습니다. 선택한 모델에 필요한 VRAM 양을 계산하려면 https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator를 확인하십시오.
)

agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run(
    "피보나치 수열의 118번째 숫자를 알려주시겠어요?",
)

</hfoption>
<hfoption id="Azure OpenAI">

Azure OpenAI에 연결하려면 `AzureOpenAIServerModel`을 직접 사용하거나 `LiteLLMModel`을 사용하고 그에 따라 구성할 수 있습니다.

`AzureOpenAIServerModel` 인스턴스를 초기화하려면 모델 배포 이름을 전달한 다음 `azure_endpoint`, `api_key` 및 `api_version` 인수를 전달하거나 환경 변수 `AZURE_OPENAI_ENDPOINT`, `AZURE_OPENAI_API_KEY` 및 `OPENAI_API_VERSION`을 설정해야 합니다.

In [ ]:
# !pip install smolagents[openai]
from smolagents import CodeAgent, AzureOpenAIServerModel

model = AzureOpenAIServerModel(model_id="gpt-4o-mini")
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run(
    "피보나치 수열의 118번째 숫자를 알려주시겠어요?",
)

마찬가지로 다음과 같이 Azure OpenAI에 연결하도록 `LiteLLMModel`을 구성할 수 있습니다.

- 모델 배포 이름을 `model_id`로 전달하고 `azure/` 접두사를 붙여야 합니다.
- 환경 변수 `AZURE_API_VERSION`을 설정해야 합니다.
- `api_base` 및 `api_key` 인수를 전달하거나 환경 변수 `AZURE_API_KEY` 및 `AZURE_API_BASE`를 설정합니다.

In [ ]:
import os
from smolagents import CodeAgent, LiteLLMModel

AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="gpt-35-turbo-16k-deployment" # 배포 이름 예시

os.environ["AZURE_API_KEY"] = "" # api_key
os.environ["AZURE_API_BASE"] = "" # "https://example-endpoint.openai.azure.com"
os.environ["AZURE_API_VERSION"] = "" # "2024-10-01-preview"

model = LiteLLMModel(model_id="azure/" + AZURE_OPENAI_CHAT_DEPLOYMENT_NAME)
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run(
   "피보나치 수열의 118번째 숫자를 알려주시겠어요?",
)

</hfoption>
<hfoption id="Amazon Bedrock">

`AmazonBedrockServerModel` 클래스는 Amazon Bedrock과의 기본 통합을 제공하여 직접 API 호출 및 포괄적인 구성을 허용합니다.

기본 사용법:

In [ ]:
# !pip install smolagents[aws_sdk]
from smolagents import CodeAgent, AmazonBedrockServerModel

model = AmazonBedrockServerModel(model_id="anthropic.claude-3-sonnet-20240229-v1:0")
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run(
    "피보나치 수열의 118번째 숫자를 알려주시겠어요?",
)

고급 구성:

In [ ]:
import boto3
from smolagents import AmazonBedrockServerModel

# 사용자 지정 Bedrock 클라이언트 만들기
bedrock_client = boto3.client(
    'bedrock-runtime',
    region_name='us-east-1',
    aws_access_key_id='YOUR_ACCESS_KEY',
    aws_secret_access_key='YOUR_SECRET_KEY'
)

additional_api_config = {
    "inferenceConfig": {
        "maxTokens": 3000
    },
    "guardrailConfig": {
        "guardrailIdentifier": "identify1",
        "guardrailVersion": 'v1'
    },
}

# 포괄적인 구성으로 초기화
model = AmazonBedrockServerModel(
    model_id="us.amazon.nova-pro-v1:0",
    client=bedrock_client,  # 사용자 지정 클라이언트 사용
    **additional_api_config
)

agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run(
    "피보나치 수열의 118번째 숫자를 알려주시겠어요?",
)

LiteLLMModel 사용:

또는 Bedrock 모델과 함께 `LiteLLMModel`을 사용할 수 있습니다.

In [ ]:
from smolagents import LiteLLMModel, CodeAgent

model = LiteLLMModel(model_name="bedrock/anthropic.claude-3-sonnet-20240229-v1:0")
agent = CodeAgent(tools=[], model=model)

agent.run("양자 컴퓨팅의 개념을 설명해주세요.")

</hfoption>
<hfoption id="mlx-lm">

In [ ]:
# !pip install smolagents[mlx-lm]
from smolagents import CodeAgent, MLXModel

mlx_model = MLXModel("mlx-community/Qwen2.5-Coder-32B-Instruct-4bit")
agent = CodeAgent(model=mlx_model, tools=[], add_base_tools=True)

agent.run("피보나치 수열의 118번째 숫자를 알려주시겠어요?")

</hfoption>
</hfoptions>

## 고급 에이전트 구성

### 에이전트 종료 조건 사용자 지정

기본적으로 에이전트는 `final_answer` 함수를 호출하거나 최대 단계 수에 도달할 때까지 계속 실행됩니다.
`final_answer_checks` 매개변수를 사용하면 에이전트가 실행을 종료하는 시기와 방법을 더 효과적으로 제어할 수 있습니다.

In [ ]:
from smolagents import CodeAgent, InferenceClientModel

# 사용자 지정 최종 답변 확인 함수 정의
def is_integer(final_answer: str, agent_memory=None) -> bool:
    """final_answer가 정수이면 True를 반환합니다."""
    try:
        int(final_answer)
        return True
    except ValueError:
        return False

# 사용자 지정 최종 답변 확인으로 에이전트 초기화
agent = CodeAgent(
    tools=[],
    model=InferenceClientModel(),
    final_answer_checks=[is_integer]
)

agent.run("3과 7의 최소 공배수를 계산하세요.")

`final_answer_checks` 매개변수는 각각 다음을 수행하는 함수 목록을 허용합니다.
- 에이전트의 final_answer 문자열과 에이전트의 메모리를 매개변수로 사용합니다.
- final_answer가 유효한지(True) 아닌지(False) 나타내는 부울 값을 반환합니다.

함수가 `False`를 반환하면 에이전트는 오류 메시지를 기록하고 실행을 계속합니다.
이 유효성 검사 메커니즘을 통해 다음을 수행할 수 있습니다.
- 출력 형식 요구 사항 적용(예: 수학 문제에 대한 숫자 답변 보장)
- 도메인별 유효성 검사 규칙 구현
- 자체 출력을 유효성 검사하는 더 강력한 에이전트 만들기

## 에이전트 실행 검사

실행 후 발생한 상황을 검사하는 데 유용한 몇 가지 속성은 다음과 같습니다.
- `agent.logs`는 에이전트의 세분화된 로그를 저장합니다. 에이전트 실행의 모든 단계에서 모든 것이 사전에 저장된 다음 `agent.logs`에 추가됩니다.
- `agent.write_memory_to_messages()`를 실행하면 에이전트의 메모리가 모델이 볼 수 있도록 채팅 메시지 목록으로 작성됩니다. 이 메서드는 로그의 각 단계를 거치며 관심 있는 내용만 메시지로 저장합니다. 예를 들어 시스템 프롬프트와 작업을 별도의 메시지에 저장한 다음 각 단계에 대해 LLM 출력을 메시지로 저장하고 도구 호출 출력을 다른 메시지로 저장합니다. 발생한 상황에 대한 상위 수준 보기를 원하지만 이 메서드에서 모든 로그가 기록되지는 않는 경우 이 방법을 사용하십시오.

## 도구

도구는 에이전트가 사용하는 원자적 함수입니다. LLM에서 사용하려면 API를 구성하고 LLM에 이 도구를 호출하는 방법을 설명하는 데 사용되는 몇 가지 속성도 필요합니다.
- 이름
- 설명
- 입력 유형 및 설명
- 출력 유형

예를 들어 [PythonInterpreterTool](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.PythonInterpreterTool)을 확인할 수 있습니다. 이 도구에는 이름, 설명, 입력 설명, 출력 유형 및 작업을 수행하는 `forward` 메서드가 있습니다.

에이전트가 초기화되면 도구 속성을 사용하여 도구 설명이 생성되고 에이전트의 시스템 프롬프트에 포함됩니다. 이를 통해 에이전트는 사용할 수 있는 도구와 그 이유를 알 수 있습니다.

### 기본 도구 상자

"toolkit" 추가 기능과 함께 `smolagents`를 설치하면 에이전트 권한 부여를 위한 기본 도구 상자가 제공되며, 초기화 시 `add_base_tools=True` 인수를 사용하여 에이전트에 추가할 수 있습니다.

- **DuckDuckGo 웹 검색***: DuckDuckGo 브라우저를 사용하여 웹 검색을 수행합니다.
- **Python 코드 인터프리터**: LLM 생성 Python 코드를 보안 환경에서 실행합니다. 이 도구는 코드 기반 에이전트가 이미 기본적으로 Python 코드를 실행할 수 있으므로 `add_base_tools=True`로 초기화하는 경우에만 [ToolCallingAgent](https://huggingface.co/docs/smolagents/main/ko/reference/agents#smolagents.ToolCallingAgent)에 추가됩니다.
- **Transcriber**: Whisper-Turbo를 기반으로 구축된 음성-텍스트 파이프라인으로 오디오를 텍스트로 변환합니다.

인수를 사용하여 호출하여 도구를 수동으로 사용할 수 있습니다.

In [ ]:
# !pip install smolagents[toolkit]
from smolagents import WebSearchTool

search_tool = WebSearchTool()
print(search_tool("현재 러시아 대통령은 누구인가요?"))

### 새 도구 만들기

Hugging Face의 기본 도구에서 다루지 않는 사용 사례에 대한 자체 도구를 만들 수 있습니다.
예를 들어, Hub에서 지정된 작업에 대해 가장 많이 다운로드된 모델을 반환하는 도구를 만들어 보겠습니다.

아래 코드로 시작합니다.

In [ ]:
from huggingface_hub import list_models

task = "text-classification"

most_downloaded_model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))
print(most_downloaded_model.id)

이 코드는 함수로 래핑하고 `tool` 데코레이터를 추가하기만 하면 도구로 빠르게 변환할 수 있습니다.
이것이 도구를 만드는 유일한 방법은 아닙니다. [Tool](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.Tool)의 하위 클래스로 직접 정의할 수 있으며, 예를 들어 무거운 클래스 속성을 초기화할 수 있는 더 많은 유연성을 제공합니다.

두 가지 옵션 모두 어떻게 작동하는지 봅시다.

<hfoptions id="build-a-tool">
<hfoption id="@tool로 함수 데코레이팅">

In [ ]:
from smolagents import tool

@tool
def model_download_tool(task: str) -> str:
    """
    이 도구는 Hugging Face Hub에서 지정된 작업의 가장 많이 다운로드된 모델을 반환합니다.
    체크포인트 이름을 반환합니다.

    Args:
        task: 다운로드 횟수를 가져올 작업입니다.
    """
    most_downloaded_model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))
    return most_downloaded_model.id

함수에는 다음이 필요합니다.
- 명확한 이름. 이름은 에이전트를 구동하는 LLM 두뇌를 돕기 위해 이 도구가 수행하는 작업을 충분히 설명해야 합니다. 이 도구는 작업에 대해 가장 많이 다운로드된 모델을 반환하므로 `model_download_tool`이라고 이름을 지정하겠습니다.
- 입력 및 출력 모두에 대한 유형 힌트
- 각 인수가 설명된 'Args:' 부분을 포함하는 설명(이번에는 유형 표시 없음, 유형 힌트에서 가져옴). 도구 이름과 마찬가지로 이 설명은 에이전트를 구동하는 LLM에 대한 사용 설명서이므로 소홀히 하지 마십시오.

이러한 모든 요소는 초기화 시 에이전트의 시스템 프롬프트에 자동으로 포함되므로 가능한 한 명확하게 만드십시오!

> [!TIP]
> 이 정의 형식은 `apply_chat_template`에 사용되는 도구 스키마와 동일하며 유일한 차이점은 추가된 `tool` 데코레이터입니다. 도구 사용 API에 대한 자세한 내용은 [여기](https://huggingface.co/blog/unified-tool-use#passing-tools-to-a-chat-template)를 참조하십시오.
</hfoption>
<hfoption id="하위 클래스 도구">

In [ ]:
from smolagents import Tool

class ModelDownloadTool(Tool):
    name = "model_download_tool"
    description = "이 도구는 Hugging Face Hub에서 지정된 작업의 가장 많이 다운로드된 모델을 반환합니다. 체크포인트 이름을 반환합니다."
    inputs = {"task": {"type": "string", "description": "다운로드 횟수를 가져올 작업입니다."}}
    output_type = "string"

    def forward(self, task: str) -> str:
        most_downloaded_model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))
        return most_downloaded_model.id

하위 클래스에는 다음 속성이 필요합니다.
- 명확한 `name`. 이름은 에이전트를 구동하는 LLM 두뇌를 돕기 위해 이 도구가 수행하는 작업을 충분히 설명해야 합니다. 이 도구는 작업에 대해 가장 많이 다운로드된 모델을 반환하므로 `model_download_tool`이라고 이름을 지정하겠습니다.
- `description`. `name`과 마찬가지로 이 설명은 에이전트를 구동하는 LLM에 대한 사용 설명서이므로 소홀히 하지 마십시오.
- 입력 유형 및 설명
- 출력 유형
이러한 모든 속성은 초기화 시 에이전트의 시스템 프롬프트에 자동으로 포함되므로 가능한 한 명확하게 만드십시오!
</hfoption>
</hfoptions>


그런 다음 에이전트를 직접 초기화할 수 있습니다.

In [ ]:
from smolagents import CodeAgent, InferenceClientModel
agent = CodeAgent(tools=[model_download_tool], model=InferenceClientModel())
agent.run(
    "Hugging Face Hub의 'text-to-video' 작업에서 가장 많이 다운로드된 모델의 이름을 알려주시겠어요?"
)

다음과 같은 로그가 표시됩니다.
```text
╭──────────────────────────────────────── 새 실행 ─────────────────────────────────────────╮
│                                                                                          │
│ Hugging Face Hub의 'text-to-video' 작업에서 가장 많이 다운로드된 모델의 이름을 알려주시겠어요? │
│                                                                                          │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ───────────────────────────────────────────╯
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0단계 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
╭─ 이 코드 실행: ───────────────────────────────────────────────────────────────────╮
│   1 model_name = model_download_tool(task="text-to-video")                               │
│   2 print(model_name)                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────╯
실행 로그:
ByteDance/AnimateDiff-Lightning

출력: 없음
[0단계: 지속 시간 0.27초| 입력 토큰: 2,069 | 출력 토큰: 60]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1단계 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
╭─ 이 코드 실행: ───────────────────────────────────────────────────────────────────╮
│   1 final_answer("ByteDance/AnimateDiff-Lightning")                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────╯
출력 - 최종 답변: ByteDance/AnimateDiff-Lightning
[1단계: 지속 시간 0.10초| 입력 토큰: 4,288 | 출력 토큰: 148]
Out[20]: 'ByteDance/AnimateDiff-Lightning'
```

> [!TIP]
> 도구에 대한 자세한 내용은 [전용 튜토리얼](https://huggingface.co/docs/smolagents/main/ko/./tutorials/tools#what-is-a-tool-and-how-to-build-one)을 참조하십시오.

## 다중 에이전트

다중 에이전트 시스템은 Microsoft의 프레임워크 [Autogen](https://huggingface.co/papers/2308.08155)과 함께 도입되었습니다.

이러한 유형의 프레임워크에서는 단일 에이전트 대신 여러 에이전트가 함께 작업하여 작업을 해결합니다.
대부분의 벤치마크에서 경험적으로 더 나은 성능을 보여줍니다. 이러한 더 나은 성능의 이유는 개념적으로 간단합니다. 많은 작업에서 모든 작업을 수행하는 시스템을 사용하는 대신 하위 작업에 단위를 특화하는 것을 선호합니다. 여기서는 별도의 도구 세트와 메모리를 가진 에이전트를 사용하면 효율적인 특성화를 달성할 수 있습니다. 예를 들어, 웹 검색 에이전트가 방문한 모든 웹 페이지의 내용으로 코드 생성 에이전트의 메모리를 채우는 이유는 무엇입니까? 별도로 유지하는 것이 좋습니다.

`smolagents`를 사용하여 계층적 다중 에이전트 시스템을 쉽게 구축할 수 있습니다.

이렇게 하려면 에이전트에 `name` 및 `description` 속성이 있는지 확인하십시오. 그런 다음 관리자 에이전트의 시스템 프롬프트에 포함되어 도구에 대해 수행하는 것처럼 이 관리되는 에이전트를 호출하는 방법을 알 수 있도록 합니다.
그런 다음 관리자 에이전트 초기화 시 `managed_agents` 매개변수에 이 관리되는 에이전트를 전달할 수 있습니다.

다음은 기본 [WebSearchTool](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.WebSearchTool)을 사용하여 특정 웹 검색 에이전트를 관리하는 에이전트를 만드는 예입니다.

In [ ]:
from smolagents import CodeAgent, InferenceClientModel, WebSearchTool

model = InferenceClientModel()

web_agent = CodeAgent(
    tools=[WebSearchTool()],
    model=model,
    name="web_search_agent",
    description="웹 검색을 실행합니다. 쿼리를 인수로 제공하십시오."
)

manager_agent = CodeAgent(
    tools=[], model=model, managed_agents=[web_agent]
)

manager_agent.run("Hugging Face의 CEO는 누구인가요?")

> [!TIP]
> 효율적인 다중 에이전트 구현에 대한 심층적인 예는 [GAIA 리더보드 상단에 다중 에이전트 시스템을 올린 방법](https://huggingface.co/blog/beating-gaia)을 참조하십시오.

## 멋진 Gradio 인터페이스에서 에이전트와 대화하고 생각을 시각화하십시오.

`GradioUI`를 사용하여 에이전트에 작업을 대화식으로 제출하고 생각 및 실행 프로세스를 관찰할 수 있습니다. 다음은 예입니다.

In [ ]:
from smolagents import (
    load_tool,
    CodeAgent,
    InferenceClientModel,
    GradioUI
)

# Hub에서 도구 가져오기
image_generation_tool = load_tool("m-ric/text-to-image", trust_remote_code=True)

model = InferenceClientModel(model_id=model_id)

# 이미지 생성 도구로 에이전트 초기화
agent = CodeAgent(tools=[image_generation_tool], model=model)

GradioUI(agent).launch()

내부적으로 사용자가 새 답변을 입력하면 `agent.run(user_request, reset=False)`로 에이전트가 시작됩니다.
`reset=False` 플래그는 이 새 작업을 시작하기 전에 에이전트의 메모리가 플러시되지 않음을 의미하며, 이를 통해 대화를 계속할 수 있습니다.

다른 에이전트 응용 프로그램에서 대화를 계속하기 위해 이 `reset=False` 인수를 사용할 수도 있습니다.

gradio UI에서 실행 중인 에이전트를 사용자가 중단하도록 허용하려면 `agent.interrupt()` 메서드를 트리거하는 버튼으로 이 작업을 수행할 수 있습니다.
이렇게 하면 현재 단계가 끝나면 에이전트가 중지된 다음 오류가 발생합니다.

## 다음 단계

마지막으로 필요에 맞게 에이전트를 구성했으면 Hub에 공유할 수 있습니다!

In [ ]:
agent.push_to_hub("m-ric/my_agent")

마찬가지로 Hub에 푸시된 에이전트를 로드하려면 도구의 코드를 신뢰하는 경우 다음을 사용하십시오.

In [ ]:
agent.from_hub("m-ric/my_agent", trust_remote_code=True)

더 자세한 사용법은 다음 튜토리얼을 확인하십시오.
- [코드 에이전트 작동 방식 설명](https://huggingface.co/docs/smolagents/main/ko/./tutorials/secure_code_execution)
- [좋은 에이전트를 구축하는 방법에 대한 이 가이드](https://huggingface.co/docs/smolagents/main/ko/./tutorials/building_good_agents).
- [도구 사용에 대한 심층 가이드](https://huggingface.co/docs/smolagents/main/ko/./tutorials/building_good_agents).